In [4]:
import sys
import os

# Get the current working directory (where the notebook is running)
notebook_dir = os.getcwd()

# Add the parent directory to the system path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '.'))
sys.path.append(parent_dir)
print(parent_dir)

c:\source\repos\multiagent-frameworks\Autogen_v0.4\multi_agent


In [5]:
from multi_agent_implementation import *
runtime = SingleThreadedAgentRuntime()
await register_agents(runtime)
runtime.start()

body=UserMessage(content='I want to know my credit card balance.', source='user', type='UserMessage') conversation_id='012aee6d-02d6-42fa-82ca-c6a90f76eb13'


c:\source\repos\multiagent-frameworks\Autogen_v0.4\multi_agent\multi_agent_implementation.py:197: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect.
  completion = await self._model_client.create(self._chat_history)


from A group_chat agent completion: {"agentType": "NeedMoreInfo", "body": "I need more information to determine if your query is related to Transify. Are you asking about your credit card balance as it pertains to your Transify account?"}
--------------------
Received by A Final Responder Agent:[UserMessage(content='I want to know my credit card balance.', source='user', type='UserMessage'), UserMessage(content='{"agentType": "NeedMoreInfo", "body": "I need more information to determine if your query is related to Transify. Are you asking about your credit card balance as it pertains to your Transify account?"}', source='GroupChatManager', type='UserMessage')]


c:\source\repos\multiagent-frameworks\Autogen_v0.4\multi_agent\multi_agent_implementation.py:108: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect.
  completion = await self._model_client.create(self._chat_history)


body=UserMessage(content='My account number is 1255.', source='user', type='UserMessage') conversation_id='012aee6d-02d6-42fa-82ca-c6a90f76eb13'
from A group_chat agent completion: {"agentType": "AccountInfo", "body": "My account number is 1255."}
--------------------
Received by A Account Info Agent Agent:[SystemMessage(content="\n        You are Transify Support Agent. Transify is an online payment platform.\n        You can provide information about the account details. \n        You need to know the 'account number' to provide the account details.\n        If the 'account number' is provided use the get_account_info tool to get the account details.\n        Format the response with 'Account Details:'.\n        If the 'account number' is not provided, ask the user to provide the 'account number'.                                                      \n\n        ", type='SystemMessage'), UserMessage(content='I want to know my credit card balance.', source='user', type='UserMessage'), 

c:\source\repos\multiagent-frameworks\venv\Lib\site-packages\autogen_core\tool_agent\_caller_loop.py:41: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect.
  response = await model_client.create(input_messages, tools=tool_schema, cancellation_token=cancellation_token)
c:\source\repos\multiagent-frameworks\venv\Lib\site-packages\autogen_core\tool_agent\_caller_loop.py:70: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect.
  response = await model_client.create(


from A group_chat agent completion: {"agentType": "IntermediateResult", "body": "The account number '1255' does not seem to exist in the Transify system. Please double-check the account number and try again."}
--------------------
Received by A Final Responder Agent:[UserMessage(content='I want to know my credit card balance.', source='user', type='UserMessage'), UserMessage(content='{"agentType": "NeedMoreInfo", "body": "I need more information to determine if your query is related to Transify. Are you asking about your credit card balance as it pertains to your Transify account?"}', source='GroupChatManager', type='UserMessage'), UserMessage(content='My account number is 1255.', source='user', type='UserMessage'), UserMessage(content='{"agentType": "AccountInfo", "body": "My account number is 1255."}', source='GroupChatManager', type='UserMessage'), AssistantMessage(content='<Context>\n Tool name is: get_account_info, \n                                tool input is: {"account_id":"12

In [6]:
async def get_answer(conversation_id, user_query):

    await runtime.publish_message(GroupChatMessage(body=UserMessage(content=user_query, source="user"), 
                                                conversation_id=conversation_id), DefaultTopicId(type="group_chat_any_topic", source=conversation_id)) 

    group_chat_result = ""
    try:
        # Wait for a message in the queue, or you can use a timeout if needed
        #group_chat_result = (await queue.get()).body.content
        # clear queue 
        async with condition:
            while conversation_id not in llm_results_dict:
                await condition.wait()

            
            group_chat_result = llm_results_dict[conversation_id].body.content
            del llm_results_dict[conversation_id]
            print(f"conversation_id: {conversation_id}")
            print("\033[35m" + "-" * 20 + "\033[0m")
    except Exception as e:
        # Handle any exception that may occur during the wait for the response
        print(f"Error retrieving message from queue: {e}")
        group_chat_result = "An error occurred while waiting for the response."

    print(group_chat_result)

In [7]:
conversation_id1 = str(uuid.uuid4())
user_query = "I want to know my credit card balance."
await get_answer(conversation_id1, user_query)

conversation_id: 012aee6d-02d6-42fa-82ca-c6a90f76eb13
--------------------
Could you please clarify if you're asking about your credit card balance in relation to your Transify account? If yes, please provide more details about your account or transactions so I can assist further.


In [8]:
user_query = "My account number is 1255."
await get_answer(conversation_id1, user_query)

conversation_id: 012aee6d-02d6-42fa-82ca-c6a90f76eb13
--------------------
It seems the account number '1255' is not found in the Transify system. Could you please double-check the account number and provide the correct one? Alternatively, if you suspect this error and need further clarification, let me know, and we can explore other ways to assist you!


In [6]:
user_query = "where do I find my account number?"
await get_answer(conversation_id1, user_query)

conversation_id: fb2c6c13-0027-4f00-8974-a1351e99aff7
--------------------
You can find your account number on your credit card statement or through the Transify app. If you need further assistance, please let me know.


In [9]:
user_query = "My account number is A1234567890."
await get_answer(conversation_id1, user_query)

conversation_id: 012aee6d-02d6-42fa-82ca-c6a90f76eb13
--------------------
Your Transify account with the account number 'A1234567890' is verified and active. The balance in your account is $150.00 USD. If you need additional details or to manage your account, you can use the following links:

- Account overview: [https://api.transify.com/v1/accounts/A1234567890](https://api.transify.com/v1/accounts/A1234567890)
- Balance details: [https://api.transify.com/v1/accounts/A1234567890/balance](https://api.transify.com/v1/accounts/A1234567890/balance)

Let me know if you need further assistance!


In [10]:
user_query = "what is my over draft limit?"
await get_answer(conversation_id1, user_query)

conversation_id: 012aee6d-02d6-42fa-82ca-c6a90f76eb13
--------------------
Thank you for your inquiry regarding the overdraft limit. Currently, based on the account details retrieved earlier, there is no specific mention of an overdraft facility tied to your Transify account. If you believe you have an overdraft arrangement or need more details, please confirm whether this is specifically linked to your Transify account, or provide more information.


In [11]:
user_query = "what is the maximum amount I can transact on my account number?"
await get_answer(conversation_id1, user_query)

conversation_id: 012aee6d-02d6-42fa-82ca-c6a90f76eb13
--------------------
The balance in your Transify account is $150.00 USD, but no information about overdraft limits or maximum transaction amounts is available in the account details retrieved. Could you clarify if you're looking for specific transaction limits tied to transfers, withdrawals, or purchases within Transify? This would help provide you with accurate assistance.


In [12]:
conversation_id2 = str(uuid.uuid4())
user_query = "I want to know my credit card balance. My account number is XYZ7890123456."
await get_answer(conversation_id2, user_query)

conversation_id: 163fae64-89c4-4ec9-b524-3e3687f202ad
--------------------
Based on the information retrieved:

Your credit card account is active and belongs to Alice Smith. Your current balances are 300.00 EUR and 100.00 USD.

If you have any additional inquiries, feel free to ask!


In [13]:
conversation_id3 = str(uuid.uuid4())
user_query = "Plan a 3 day road trip to zion national park"
await get_answer(conversation_id3, user_query)

conversation_id: 879b91e2-284e-4edf-9eef-e0bce103fb1a
--------------------
This query about planning a road trip to Zion National Park is not related to Transify. If you have any questions or concerns regarding Transify's services or features, feel free to share them, and I'll be happy to assist you!


In [14]:
conversation_id4 = str(uuid.uuid4())
user_query = "What are payment reversals and how to avoid them"
await get_answer(conversation_id4, user_query)

conversation_id: 95889ea0-207d-4b6c-9049-e8ce81daeb34
--------------------
Based on the conversation history, the user's question has been identified ("What are payment reversals and how to avoid them") but no direct answer was provided. Here is the complete response:

---

**Payment Reversals:** A payment reversal, also known as a chargeback or refund, occurs when funds from a payment transaction are returned to the payer. Common reasons for payment reversals include disputes raised by customers, fraudulent transactions, duplicate charges, or errors in transaction processing.

**How to Avoid Payment Reversals:**
1. **Ensure Transparent Communication:** Clearly communicate your terms and policies, including refund, cancellation, and return policies.
2. **Provide Accurate Details:** Maintain accurate product or service descriptions to avoid customer confusion.
3. **Use Fraud Protection Tools:** Identify and block potential fraudulent transactions with security tools and verification mea

In [15]:
user_query = "get my transaction details"
await get_answer(conversation_id1, user_query)

conversation_id: 012aee6d-02d6-42fa-82ca-c6a90f76eb13
--------------------
Your recent transaction was successfully completed. Here are the details:

- **Transaction ID**: 9HX25435AB0123456
- **Status**: Completed  
- **Total Amount**: $75.00 USD  
  - **Transaction Fee**: $2.50 USD  
  - **Purchased Items**:  
    1. Watercolor Paint Set - $50.00 USD  
    2. Brush Set - $25.00 USD  
  - **Shipping Cost**: $5.00 USD  

The items were shipped to the following address:  
123 Main St, Anytown, CA, 90210, US.  

Payment was made via Transify. If you'd like more information, you can view the transaction details using the following link: [Transaction Details](https://api.transify.com/v1/payments/transaction/9HX25435AB0123456).  

Let me know if you need assistance with anything else!


In [16]:
conversation_id5 = str(uuid.uuid4())
user_query = "get my transaction details my account number is A1234567890"
await get_answer(conversation_id5, user_query)

conversation_id: fe011b59-23c4-4c26-a752-c49fe171bf3e
--------------------
Here are the details of your transaction associated with account number **A1234567890**:

- **Transaction ID**: 9HX25435AB0123456  
- **Status**: Completed  
- **Total Amount**: $75.00 USD  
- **Transaction Fee**: $2.50 USD  
- **Create Time**: 2024-09-29T12:45:30Z  
- **Update Time**: 2024-09-29T12:46:45Z  

### Payer Details:
- **Name**: John Doe  
- **Email**: buyer@example.com  
- **Country**: US  

### Payee Details:
- **Merchant ID**: MERCHANT987654321  
- **Name**: Alice's Art Supplies  
- **Email**: seller@example.com  

### Payment Method:
- Transify  

### Transaction Items:
1. **Watercolor Paint Set**  
   - Quantity: 1  
   - Price: $50.00 USD  

2. **Brush Set**  
   - Quantity: 1  
   - Price: $25.00 USD  

### Shipping Information:
- **Recipient Name**: John Doe  
- **Address**: 123 Main St, Anytown, CA, 90210, US  
- **Shipping Cost**: $5.00 USD  

You can also view the transaction details at the